# How do drug manufacturer's payments affect prescription rates?

Description of the problem here:

# Loading the data

In [1]:
#imports
import pandas as pd
import pickle

##  Loading Payments Data

In [2]:
payments_types = {'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID': 'str',
 'Charity_Indicator': 'str',
 'City_of_Travel': 'str',
 'Contextual_Information': 'str',
 'Country_of_Travel': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological1': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological2': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological3': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological4': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological5': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply1': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply2': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply3': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply4': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply5': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological1': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological2': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological3': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological4': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological5': 'str',
 'Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value': 'str',
 'Physician_License_State_code2': 'str',
 'Physician_License_State_code3': 'str',
 'Physician_License_State_code4': 'str',
 'Physician_License_State_code5': 'str',
 'Physician_First_Name': 'str',
 'Physician_Last_Name': 'str',
 'Physician_Middle_Name': 'str',
 'Physician_Name_Suffix': 'str',
 'Physician_Profile_ID': 'str',
 'Recipient_Postal_Code': 'str',
 'Recipient_Primary_Business_Street_Address_Line2': 'str',
 'Recipient_Province': 'str',
 'Recipient_Zip_Code': 'str',
 'Record_ID': 'str',
 'State_of_Travel': 'str',
 'Teaching_Hospital_ID': 'str',
 'Teaching_Hospital_Name': 'str',
 'Third_Party_Equals_Covered_Recipient_Indicator': 'str'}

In [3]:
payments = pd.read_csv(r'data_unzipped/payments_14/OP_DTL_GNRL_PGYR2014_P06302016.csv', dtype=payments_types)

/Users/zakscholl/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Wrangling

In order to tie the payments data to prescribers, I had to look up the National Provider Identifier (NPI) for each physician in the payments dataset. Every prescriber in the Medicaid Part D prescriptions dataset has an NPI associated with them, but because of the law written by congress, the NPI is not to be released with the payments data. 

I used the National Plan and Provider Enumeration System's (NPPES) NPI [registry](https://npiregistry.cms.hhs.gov/) to lookup each physician's NPI. The code I used to do this is listed below, but it is not included in the live notebook because of memory constraints. The NPPES full dataset is over 5.7GB.



```python
physician_info = payments[['Physician_Profile_ID', 'Physician_First_Name', 'Physician_Middle_Name',
       'Physician_Last_Name', 'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_City',
       'Recipient_State']]
       
nppes = pd.read_csv(r'data_unzipped/NPPES/npidata_20050523-20170108.csv', 
                    usecols=['NPI', 'Provider Last Name (Legal Name)','Provider First Name', 
                             'Provider Middle Name', 
                             'Provider First Line Business Mailing Address',
                             'Provider Business Mailing Address City Name', 
                             'Provider Business Mailing Address State Name'], 
                    dtype={'Provider Business Mailing Address Postal Code': 'str'})
                    
# change the column names to match for merging
nppes.columns = ['NPI', 'last_name', 'first_name', 'middle_name', 'address', 'city', 'state']
physician_info.columns = ['Physician_Profile_ID', 'first_name', 'middle_name', 'last_name', 'address', 
                          'city', 'state']
                          
def safe_upper(x):
    """
    Transforms an object to an uppercase string or returns the object
    in the case it is not a string.
    """
    try:
        return x.upper()
    except AttributeError:
        return x

# make all the fields into uppercase
physician_info['first_name'] = physician_info['first_name'].apply(lambda x: safe_upper(x))
physician_info['last_name'] = physician_info['last_name'].apply(lambda x: safe_upper(x))
physician_info['middle_name'] = physician_info['middle_name'].apply(lambda x: safe_upper(x))
physician_info['address'] = physician_info['address'].apply(lambda x: safe_upper(x))
physician_info['city'] = physician_info['city'].apply(lambda x: safe_upper(x))


# drop any duplicate entries in the physicians info
physician_info = physician_info.drop_duplicates()

physician_npi = physician_info.merge(nppes, on=['last_name', 'first_name', 
                                                'middle_name', 'address', 
                                                'city', 'state'], how='inner')
# Drop any duplicates from the resulting merge
physician_npi = physician_npi[~physician_npi.duplicated(subset=['last_name', 'first_name', 
                                                                'middle_name','address', 
                                                                'city', 'state'] ,keep=False)]
                                                                
physician_id_to_npi = physician_npi[['Physician_Profile_ID', 'NPI']]

```

With this lookup we can find an NPI for 44% of the Physicians in the payments dataset. This isn't ideal, but it should give us a good cross-section on the prescription dataset.

I have this mapping saved as a pickled object, so for the live notebook all I have to do is load it using pickle.

In [4]:
file_in = open(r'physician_id_to_npi.pkl', 'rb')

physician_id_to_npi = pickle.load(file_in)

file_in.close()

## Load Prescription Data

In [5]:
prescriptions = pd.read_csv(r'data_unzipped/PartD_14/PartD_Prescriber_PUF_NPI_Drug_14.txt', sep='\t')

## Load Drug Manufacturer Data

In [6]:
drug_manufacturers = pd.read_csv(r'drug_manufacturers_cleaned.csv')

In [7]:
drug_manufacturers.head(1)

,brand_name,generic_name,manufacturer_name,cleaned
0,VARICOGO,"AESCULUS HIPPOCASTANUM, ARNICA MONTANA, CALCAR...","Native Remedies, LLC",native remedies llc


### Drug manufacturers coverage

We need to be able to map each drug manufacturer in the payments dataset to the drugs that they manufacture. In order to do that I have used the Food and Drug Administration's [drug labels dataset.](https://open.fda.gov/drug/label/) I downloaded the entire json dataset and extracted the drug brand name, generic name, and manufacturer from every label and saved it to the csv file I loaded in above.

By naively matching the drug manufacturers in the payments dataset on the manufacturer name we can cover 37% of the payments. If we force lowercase and remove any punctuation from the manufacturers in the payments dataset and the manufacturers in the drug labels dataset, then we can more than double our coverage to 77%.


In [16]:
len(payments[
        payments.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name.isin(
            drug_manufacturers.manufacturer_name)]) / float(len(payments))

0.3782588782806642

In [9]:
import string
len(payments[
        payments.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name.str.lower().str.translate(None, string.punctuation).isin(
            drug_manufacturers.cleaned)]) / float(len(payments))

0.7700761754060829

### Prescriptions with NPI coverage

It's also important that we get a significant cross-section of providers from the prescriptions dataset and the payments dataset.

To measure that we will see how many NPIs in the prescriptions dataset we came up with in the NPI to physician ID mapping table.

For 36% of the prescription records we have information on that provider's payments from drug manufacturers. I don't know what percentage of all physicians receive payments from drug manufacturers so I can't quantify what proportion of physicians we aren't able to tie their prescriptions to their payments, but 36% of all prescription records from 2014 is a good sample size.

In [10]:
len(prescriptions[prescriptions.NPI.isin(physician_id_to_npi.NPI)]) / float(len(prescriptions))

0.36248128704580396

# Predicting prescription rates

Here's a dilemma: can I compare those who do not have payment records with those who do? Because I can't confidently say that if I don't have an NPI to physician ID mapping then that provider didn't recieve any payments from any drug manufacturers, I don't think I can compare these two groups.

So what I may have to do instead is use the group of providers who I can tie to drug manufacturer payments and compare between those who recieved little or no payment from a given manufacturer and those who recieved more payment. I can be more confident that the difference between these groups is more significant.

I also can include the provider's specialty and location from the prescription dataset to offset any differences in patient populations that impact the prescription rates of drugs.

In [18]:
prescriptions[prescriptions.NPI.isin(physician_id_to_npi.NPI)].head()

,NPI,NPPES_PROVIDER_LAST_ORG_NAME,NPPES_PROVIDER_FIRST_NAME,NPPES_PROVIDER_CITY,NPPES_PROVIDER_STATE,SPECIALTY_DESCRIPTION,DESCRIPTION_FLAG,DRUG_NAME,GENERIC_NAME,BENE_COUNT,TOTAL_CLAIM_COUNT,TOTAL_DAY_SUPPLY,TOTAL_DRUG_COST,BENE_COUNT_GE65,BENE_COUNT_GE65_SUPPRESS_FLAG,TOTAL_CLAIM_COUNT_GE65,GE65_SUPPRESS_FLAG,TOTAL_DAY_SUPPLY_GE65,TOTAL_DRUG_COST_GE65
73,1588763981,AABERG,RANDAL,HONOLULU,HI,Urology,S,ANDROGEL,TESTOSTERONE,NaN,17,720,10412.06,NaN,*,17.0,NaN,720.0,10412.06
74,1588763981,AABERG,RANDAL,HONOLULU,HI,Urology,S,BICALUTAMIDE,BICALUTAMIDE,26.0,80,6000,3538.82,NaN,#,NaN,#,NaN,NaN
75,1588763981,AABERG,RANDAL,HONOLULU,HI,Urology,S,CEPHALEXIN,CEPHALEXIN,24.0,24,160,262.86,24.0,NaN,24.0,NaN,160.0,262.86
76,1588763981,AABERG,RANDAL,HONOLULU,HI,Urology,S,CIPROFLOXACIN HCL,CIPROFLOXACIN HCL,52.0,63,415,511.65,NaN,#,NaN,#,NaN,NaN
77,1588763981,AABERG,RANDAL,HONOLULU,HI,Urology,S,FINASTERIDE,FINASTERIDE,124.0,354,28140,31082.02,NaN,#,NaN,#,NaN,NaN


### Form of data

Thinking about what form I want this dataset in to make predictions or build a regression model of the total days supply and number of claims filed for each drug by each provider.

| Provider      | Drug                  | Number of Claims  | Total Day Supply     | Specialty (one-hot?) | Amount of payment   | Location (CityState) one-hot? |  
| ------------- |---------------------- | ----------------- | -------------------- | -------------------- | ------------------- | ----------------------------- |
|    124        |   Abilify             |        4          |     25               |  Urology             |     \$200.45        |        New York               |
|    123        |   Tylenol-Hydrocodone |        62         |     200              |  Family Medicine     |     NaN             |        San Diego              |

In this scenario I want to build two models, one with Number of Claims as my output or Y variable and one with Total Day Supply as my output/Y variable. 

My hypotheses are that payments from drug manufacturers influence the total number of claims filed for that drug, and that they also influence the the total day supply of the claims that were filed. 

If there is something suspicious going on it would probably present itself in the total day supply. If a physician is prescribing an unusually high volume of supply for the number of claims when they are recieving payments from a manufacturer, that certainly looks suspicous to me. 

With the data in this form I can make scatter plots comparing the amount of payment to both the number of claims and total day supply. I can color code based on those that have payment records for those drugs and those that do not. This could reveal a possible difference.